# NOTEBOOK FOURRE-TOUT pour scrips PlaMADE

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import os, re
import pandas as pd
import geopandas as gp
import zipfile
import Connexion_Transfert as ct
import Outils as O

In [ ]:
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees'
#dézipper tous les dossier de Gérard
for root,dirs, files in os.walk(dossierSrc) : 
    for f in files : 
        if f.endswith('.zip') : 
            cheminFichier=os.path.join(root, f)
            print(cheminFichier[:-4])
            try:
                with zipfile.ZipFile(cheminFichier) as z:
                    z.extractall(cheminFichier[:-4])
                    print(f"fichier extrait {cheminFichier[:-4]}")
            except:
                print(f"pb extraction sur dossier {cheminFichier[:-4]}")

## creer une bdd des fichiers geostandardises

In [3]:
#pour traiter tous les fichiers d'un dossier général.suppose que la structure des tables a déjà été crée (par exemple avec l'import des des fichiers pouis truncate)
dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees'
coupleFichierTable=(('N_ROUTIER_ALLURE','allure_national'),('N_ROUTIER_REVETEMENT','rvt_national'),('N_ROUTIER_ROUTE','route_national'),
                    ('N_ROUTIER_TRAFIC','trafic_national'),('N_ROUTIER_VITESSE','vts_national'))
#dossierSrc=r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne-Rhone-Alpes\Donnees_geostandardisees\Route01_v2_dec2020'
listEreur=[]
with ct.ConnexionBdd('local_PlaMADE', 'boulot') as c :
    for root,dirs, files in os.walk(dossierSrc) : 
        for f in files :
            if f.endswith(('.shp', '.dbf')) : 
                print(os.path.join(root,f))
                if f.endswith('.shp') and 'N_ROUTIER_TRONCON' in f :
                    try :
                        ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID=None,geotype='MULTILINESTRING', dims=3, creationMode='-append -update',encodageClient='UTF-8', version_simple=True)
                    except Exception as e: 
                        listEreur.append({'fichier': f, erreur : e})
                else : 
                    for fich,t in coupleFichierTable :
                        try : 
                            if fich in f.upper() and f.endswith('.dbf') : 
                                ct.ogr2ogr_shp2pg(c.connstringOgr,os.path.join(root,f),
                                                  schema='geostandardise_src', table=t, SRID=None,geotype=None, dims=None, creationMode='-append -update',encodageClient='UTF-8', requeteSql='', version_simple=True)
                        except Exception as e: 
                            listEreur.append({'fichier': f, 'erreur' : e})

C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf
debut import fichier C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf avec shape2pg à 15:17:52 
 avec commande ogr2ogr -append -update -f "postgreSQL" -lco "SCHEMA=geostandardise_src" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=plamade user=postgres password=postgres port=5432" C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_ALLURE_001.dbf -nln geostandardise_src.allure_national
Fait
C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Auvergne_Rhone_Alpes\Donnees_geostandardisees\Route01_v2_dec2020\N_ROUTIER_REVETEMENT_001.dbf
debut import fichier C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_g

In [6]:
#pour test sur un dept
with ct.ConnexionBdd('local_PlaMADE', 'boulot') as c :
    ct.ogr2ogr_shp2pg(c.connstringOgr,r'C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne\35\N_ROUTIER_TRONCON_L_035.shp',
                                              schema='geostandardise_src', table='troncon_national',
                                              SRID='2154',geotype='MULTILINESTRING', dims=3, creationMode='-append -update',encodageClient='UTF-8')

debut import fichier C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne\35\N_ROUTIER_TRONCON_L_035.shp avec shape2pg à 17:23:54 
 avec commande ogr2ogr -append -update -f "postgreSQL" --config PG_USE_COPY YES -a_srs "EPSG:2154"  -nlt MULTILINESTRING -dim 3 -lco "SCHEMA=geostandardise_src" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=plamade user=postgres password=postgres port=5432" C:\Users\martin.schoreisz\Box\Donnees_source\Donnees_geostandardisees\Bretagne\35\N_ROUTIER_TRONCON_L_035.shp -nln geostandardise_src.troncon_national 
Fait


In [ ]:
#transfert des deparyemenst
with ct.ConnexionBdd('local_PlaMADE', 'boulot') as c :
    for d in range(1,95) : 
        if d==20 : 
            continue
        dept=f'0{d}' if d<10 else str(d)
        cheminDept = fr'Y:\REF_GEO\BD_Topo\D{dept}\ED19\SHP\1_DONNEES_LIVRAISON\ADMINISTRATIF\N_DEPARTEMENT_TOPO_0{dept}.shp'
        if d==1 : 
            ct.ogr2ogr_shp2pg(c.connstringOgr,cheminDept,
                                                  schema='geostandardise_src', table='dept', SRID='2154',geotype='MULTIPOLYGON', dims=2, creationMode='',encodageClient='UTF-8', requeteSql='')
        else :
            ct.ogr2ogr_shp2pg(c.connstringOgr,cheminDept,
                                                  schema='geostandardise_src', table='dept', SRID='2154',geotype='MULTIPOLYGON', dims=2, creationMode='-append -update',encodageClient='UTF-8', requeteSql='')

In [88]:
O.epurationNomRoute('A000602Bretelle')

'A602Bretelle'

In [82]:
nomRoute='D603'
i,j=0,0 #initialisation compteur
while nomRoute[i].isalpha(): #determiner le prefixe
    i+=1
prefixe=nomRoute[:i] if nomRoute[:i] not in ('RN', 'RD') else nomRoute[i-1]
suffixe=nomRoute[i:]
while suffixe[j]=='0': #determine le nombre de 0 non significatif
    j+=1
suffixeTraite=suffixe[j:]
''.join(re.search('([A,D,N])([0-9]{1,4})([A-Z]{1,3})(0{1,5})([0-9]{1,5})',prefixe+suffixeTraite).group(1,2,3,5))

AttributeError: 'NoneType' object has no attribute 'group'

In [76]:
''.join(re.search('([A,D,N])([0-9]{1,4})([A-Z]{1,3})(0{1,5})([0-9]{1,5})','D602E066').group(1,2,3,5))

'D602E66'